In [1]:
import logging.config
from typing import List
from dotenv import load_dotenv
from fastapi import FastAPI, Query, Request, Response, Depends, Header
from fastapi.responses import JSONResponse
from db.db_conf import get_db
from fastapi.middleware.cors import CORSMiddleware
from sqlalchemy.orm import Session
from models.user import User
import jwt
from logs.logs_config import LOGGING_CONFIG
import logging
from schemas.response_schemas.secured import SecuredResponse
import json
from fastapi import FastAPI
from sqlalchemy.orm import Session
import os
from dotenv import load_dotenv
from fastapi_utils.session import FastAPISessionMaker

import requests 
from models import Order, Promo, PromoStats, Stats, Stocks, KeyWordsStats, Product
import datetime
import pandas as pd
from time import sleep
from sqlalchemy import desc

load_dotenv()
from __future__ import print_function
from collections import defaultdict
from sqlalchemy.inspection import inspect

database_uri = os.environ.get('DATABASE_URL')

# session = Session()
sessionmaker = FastAPISessionMaker(database_uri)

def query_to_dict(rset):
    result = defaultdict(list)
    for obj in rset:
        instance = inspect(obj)
        for key, x in instance.attrs.items():
            result[key].append(x.value)
    return result

with sessionmaker.context_session() as db:
    try:
       stats = db.query(Stats).select_from(Stats).join(Product).join(User).all()
       df = pd.DataFrame(query_to_dict(stats))
    except Exception as e:
        print(e)
        print(e)
df



,id,date,nmId,product_id,vendorCode,brandName,openCardCount,addToCartCount,ordersCount,ordersSumRub,buyoutsCount,buyoutsSumRub,cancelCount,cancelSumRub,addToCartPercent,cartToOrderPercent,buyoutsPercent,avg_price_rub,user_id
0,11171,2025-04-12,178177547,38,WB1001,Home's Factor,7,0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,1
1,11172,2025-04-12,183634312,39,WB1010,Home's Factor,0,0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,1
2,11173,2025-04-12,186940293,36,WB1011,Home's Factor,8,0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,1
3,11175,2025-04-12,233096759,37,WB1015,Home's Factor,2,0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,1
4,11176,2025-04-12,251665957,27,WM300,Home's Factor,0,0,0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,12166,2025-03-19,179414402,12,WB1004,Home's Factor,398,69,32,28800.0,26,23400.0,2,1800.0,17.0,46.0,93.0,900.0,1
1088,12167,2025-03-19,261667902,7,WB1017,Home's Factor,603,45,13,28860.0,9,19980.0,1,2220.0,7.0,29.0,90.0,2220.0,1
1089,12168,2025-03-19,200263912,6,WB1013,Home's Factor,483,22,9,40050.0,8,35600.0,0,0.0,5.0,41.0,100.0,4450.0,1
1090,12169,2025-03-19,261667572,5,WB1016,Home's Factor,576,46,17,50843.0,12,35783.0,1,3020.0,8.0,37.0,92.0,2991.0,1


In [ ]:
df.to_excel('df_pavel.xlsx')

In [ ]:
from catboost import CatBoostRegressor



In [ ]:
df

### 1. Обрабатываем фичи

In [ ]:
df.dtypes

In [ ]:
# выделяем год, месяц, день недели и убираем колонку с полной датой
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day_of_week'] = df['date'].dt.dayofweek
df = df.drop(columns='date')
df.head()

In [ ]:
# one hot encoding категориальных фичей, чтобы модель могла их обработать
df = pd.get_dummies(df, columns=['vendorCode', 'brandName'])

### 2) Создаем тренировочный и тестовый датасеты

In [ ]:
# делим на матрицу признаков и таргет (число заказов)
X = df.drop(columns='ordersCount')
y = df['ordersCount']

In [ ]:
# делим на тренировочную и тестовую выборку
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

### 3) Обучаем модель градиентного бустинга CatBoost

In [ ]:

from catboost import CatBoostRegressor

In [53]:
from models.warehouses import Warehouse
from models.product import Product
from typing import List
from sqlalchemy.sql import func

nmId = '200263560'

# with sessionmaker.context_session() as db:
    
def count_storage(item: Product):
    whs =  db.query(Stocks).filter(Stocks.nmId==item.wb_article).all()
    storage_sum = 0 
    for w in whs:
        warehouse_tariff: Warehouse = db.query(Warehouse).filter(Warehouse.warehouseName==w.warehouseName).first()
        if warehouse_tariff:
            storage_sum_one = float(warehouse_tariff.boxStorageBase.replace(',','.'))+ (item.volume-1)*float(warehouse_tariff.boxStorageLiter.replace(',','.'))

            warehouses_total_storage = storage_sum_one*w.quantity

            storage_sum+=warehouses_total_storage
    
    return storage_sum
        
# def count_liters(item: Product):
#     print(item.volume)

def count_delivery(item: Product, warehouse: Warehouse):
    logists_cost  = float(warehouse.boxDeliveryBase.replace(',','.')) + (item.volume-1)*float(warehouse.boxDeliveryLiter.replace(',','.'))
    return logists_cost

def count_full_delivery_cost(delivery_cost,buyoutsPercent):
    return delivery_cost + 50 * (1 - buyoutsPercent / 100)


# def count_promo_cost(stats: PromoStats):
#     return
    
    

def calculate_margin(nmId):
    with sessionmaker.context_session() as db:
        user = db.query(User).first()
        item_warehouse = db.query(Stocks).filter(Stocks.nmId==nmId).order_by(desc(Stocks.quantity)).limit(1).first()
        item: Product = db.query(Product).filter(Product.wb_article==nmId).first()

        storage_sum = count_storage(item)

        # print(storage_sum)
        # print(item.volume)
        # print(item.length)
        # print(warehouse.warehouseName)

        now = datetime.datetime.today().strftime('%Y-%m-%d')
        warehouse_tariff = db.query(Warehouse).filter(Warehouse.warehouseName==item_warehouse.warehouseName).first()

        delivery_cost = count_delivery(item, warehouse_tariff)
        backwards_cost = 50
        buyoutsPercent = db.query(Stats).filter(Stats.nmId==nmId).order_by(desc(Stats.date)).limit(1).first()
        
        if not buyoutsPercent:
            buyoutsPercent=0.9
        else:
            if buyoutsPercent.buyoutsPercent < 70:
                avg = db.query(func.avg(Stats.buyoutsPercent).label('average')).filter(Stats.nmId==nmId).first()
                buyoutsPercent = avg.average

        
        # print(buyoutsPercent.date)
                
        full_delivery_cost = 0

        full_delivery_cost = count_full_delivery_cost(delivery_cost,buyoutsPercent)


        promo_stats: PromoStats = db.query(PromoStats).filter(PromoStats.nmId==nmId).order_by(desc(PromoStats.date)).limit(1).first()
        promo_cost = promo_stats.sum
        # print(promo_cost)
        profit = item.wb_price  - item.cost_price - full_delivery_cost - (item.cost_price*item.commision_percentage) - storage_sum - promo_cost# добавить рекаму


        margin = profit/item.wb_price*
        print(profit, round(margin,2)*100)

        return margin, profit


print(calculate_margin(nmId))



2009.6328571428573 62.0
(0.6189198820889613, 2009.6328571428573)


In [ ]:
from models.warehouses import Warehouse

with sessionmaker.context_session() as db:
    user = db.query(User).first()
    warehouse = db.query(Stocks).filter(Stocks.nmId==nmId).order_by(desc(Stocks.quantity)).limit(1).first()
    print(warehouse.warehouseName)

    now = datetime.datetime.today().strftime('%Y-%m-%d')
    print(now)
    warehouses = requests.get(f"https://common-api.wildberries.ru/api/v1/tariffs/box?date={now}", 
        headers={
        'Authorization': user.token
    }).json()

    for w in warehouses['response']['data']['warehouseList']:
        warehouse = Warehouse()
        warehouse.boxDeliveryAndStorageExpr = w['boxDeliveryAndStorageExpr']
        warehouse.boxDeliveryBase =  w['boxDeliveryBase']
        warehouse.boxDeliveryLiter =  w['boxDeliveryLiter']
        warehouse.boxStorageBase =  w['boxStorageBase']
        warehouse.boxStorageLiter =  w['boxStorageLiter']
        warehouse.warehouseName =  w['warehouseName']

        db.add(warehouse)
        db.commit()
    